In [ ]:
import tweepy
import time
import csv
import os
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive')

In [ ]:
df = pd.read_parquet('final_twitter_data.parquet')
df.drop_duplicates(subset=['tweet_id'])

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

In [ ]:
def extract_most_frequent_hashtags(dataset):
    dates=list()

    for row,record in enumerate(df.hashtags):
      if record!='[]'and df['timestamp'][row][0:7]>'2021-05' : 
        dates.append(df['timestamp'][row][0:7])
    date=list()
    for dt in dates:
      if dt not in date:
        date.append(dt)




    i=1
    j=6
    for i in range(1,3):
      for j in range(1,13):
        if i==1 and j>5 and j<10 or i==2 and j<10:
          globals()[f'hash202{i}_0{j}']=list()
          globals()[f'hash202{i}_0{j}'].extend([hash for row,hash in enumerate(df['hashtags']) if df['timestamp'][row][0:7]==f'202{i}-0{j}' and hash!='[]'])
            
        elif i==1 and j>5 and j>=10 or i==2 and j>=10:
          globals()[f'hash202{i}_{j}']=list()
          globals()[f'hash202{i}_{j}'].extend([hash for row,hash in enumerate(df['hashtags']) if df['timestamp'][row][0:7]==f'202{i}-{j}' and hash!='[]'])
          
    #preprocessing the hashtags list from specific periods
    i=1
    j=6
    for i in range(1,3):
      for j in range(1,13):
        if i==1 and j>5 and j<10 or i==2 and j<10:
          globals()[f'all_hashtags_202{i}_0{j}']=list()
          for s in globals()[f'hash202{i}_0{j}']:
              bb=s.split(', ')
              a=[re.search(r"[\[\']*(\w*)[\]\']*",i).group(1) for i in bb]
              globals()[f'all_hashtags_202{i}_0{j}'].extend(a)
            
            
        elif i==1 and j>5 and j>=10 or i==2 and j>=10:
          globals()[f'all_hashtags_202{i}_{j}']=list()
          for s in globals()[f'hash202{i}_{j}']:
              bb=s.split(', ')
              a=[re.search(r"[\[\']*(\w*)[\]\']*",i).group(1) for i in bb]
              globals()[f'all_hashtags_202{i}_{j}'].extend(a)


    def hashtag_set_finder():
      hashtag_set=set()
      for dt in date:
          i,j = dt[3],dt[5:]
          lis = globals()[f'all_hashtags_202{i}_{j}']
          for i in lis:
            hashtag_set.add(i)
      return hashtag_set
    hashtags_in_DS=hashtag_set_finder()
    hashtags_in_DS2 = {i.lower() for i in hashtags_in_DS}
    def generate_hashtags(dataset):
      listOfAllHashtags=list()
      for row, hash in enumerate(dataset.hashtags):
        if dataset['timestamp'][row][0:10]>="2021-06-01" and hash!='[]':
          bb=hash.split(', ') 
          
          a=[re.search(r"[\[\']*(\w*)[\]\']*",i).group(1) for i in bb]
          listOfAllHashtags.extend(a)
      listOfAllHashtagsL=[i.lower() for i in listOfAllHashtags]
      return listOfAllHashtagsL
    list_of_all_hashtags = generate_hashtags(df)
    dic_of_mfh={i:list_of_all_hashtags.count(i)/len(list_of_all_hashtags) for i in hashtags_in_DS2 if list_of_all_hashtags.count(i)>500}
      #input_hashtags = [i[0] for i in list_of_mfh]
    return dic_of_mfh

In [ ]:
def hashtag_groups():
    model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')
    dic_of_mfh= extract_most_frequent_hashtags(df)
    input_hashtags=list(dic_of_mfh.keys())
    sentence = input_hashtags
    embedding_full = model.encode(sentence)
    clusterd_hashtags=list()
    for hash in input_hashtags:
      embedding = model.encode(hash)
      sublist={hash}
      for j in range(len(input_hashtags)):
        if j<len(input_hashtags)-1:
          if util.pytorch_cos_sim(embedding,model.encode(input_hashtags[j+1])) > 0.5:
              sublist.add(input_hashtags[j+1])
              input_hashtags.remove(input_hashtags[j+1])
      clusterd_hashtags.append(sublist)

      clusterd_hashtags.sort(reverse=True, key= lambda i: len(i))
    print("for each cluster a group name has been extracted based on the most frequent hashtag of that group or a Group Representative\n")
    for i in clusterd_hashtags:
      i=list(i)
      group_n=i[0]
      for j in i:
        if dic_of_mfh[j] > dic_of_mfh[group_n]:
          group_n=j
      print(f"group name  :  {group_n}" )
      print(f'{i}\n') 

In [ ]:
hashtag_groups()

for each cluster a group name has been extracted based on the most frequent hashtag of that group or a Group Representative

group name  :  wisszeitvg
['woisthanna', 'tvstud', 'wisskomm', 'karliczek', 'wisszeitvg', 'ichwarhanna']

group name  :  wissenschaft
['wissenschaft', 'bundestag', 'daad', 'postdocs', 'hannainzahlen']

group name  :  berlhg
['ichbinhannach', 'berlhg', 'bmbf']

group name  :  hochschulen
['academia', 'hochschule', 'hochschulen']

group name  :  dauerstellen
['dauerstellen', 'dauerstellenfürdaueraufgaben', 'befristung']

group name  :  academictwitter
['twittercampus', 'academictwitter']

group name  :  hannabeidergew
['hannastreikt', 'hannabeidergew']

group name  :  27juni
['27juni']

group name  :  hannaorganisiertsich
['hannaorganisiertsich']

group name  :  evaluation
['evaluation']

group name  :  acertaindegreeofflexibility
['acertaindegreeofflexibility']

group name  :  tenuretrack
['tenuretrack']

group name  :  corona
['corona']

group name  :  forschung
